In [1]:
#import python classes and packages
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
import os
from keras.layers import Dense
from keras_dgl.layers import GraphCNN #loading Graph Neural Network class
import keras.backend as K
from keras.regularizers import l2
import pandas as pd
import numpy as np
from mitreattack.stix20 import MitreAttackData #loading mitre tool API to calculate score
import seaborn as sns
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
c:\users\bunny yadav\appdata\local\programs\python\python37\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\bunny yadav\appdata\local\programs\python\python37\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\bunny yadav\appdata\local\programs\python\python37\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qi

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'OpenSSL 1.1.0j  20 Nov 2018'. See: https://github.com/urllib3/urllib3/issues/2168

In [4]:
#loading mitre attack analysis tool with enterprise attack data
mitre_attack_data = MitreAttackData("enterprise-attack.json")
print("Mitre Loaded")

NameError: name 'MitreAttackData' is not defined

In [3]:
#loading and displaying darpa 2000 attack dataset
dataset = pd.read_csv("Dataset/Darpa.csv")
dataset

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,175.45.176.3,21223,149.171.126.18,32780,udp,INT,0.000021,728,0,254,...,0,1,1,1,1,1,1,1,Exploits,1
1,175.45.176.2,23357,149.171.126.16,80,tcp,FIN,0.240139,918,25552,62,...,0,3,2,2,1,1,1,1,Exploits,1
2,175.45.176.0,13284,149.171.126.16,80,tcp,FIN,2.390390,1362,268,254,...,0,5,2,2,1,1,1,1,Reconnaissance,1
3,175.45.176.2,13792,149.171.126.16,5555,tcp,FIN,0.175190,8168,268,254,...,0,1,1,1,1,1,1,1,Exploits,1
4,175.45.176.2,26939,149.171.126.10,80,tcp,FIN,0.190600,844,268,254,...,0,3,1,1,1,1,1,1,Exploits,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22210,175.45.176.1,58463,149.171.126.15,179,tcp,FIN,31.461020,924,708,254,...,0,6,6,2,2,2,1,2,Fuzzers,1
22211,175.45.176.1,58460,149.171.126.15,179,tcp,FIN,57.995934,1362,536,254,...,0,6,6,2,2,2,1,2,Fuzzers,1
22212,175.45.176.1,58967,149.171.126.15,179,tcp,FIN,32.361782,1128,622,254,...,0,6,6,2,2,2,1,2,Fuzzers,1
22213,175.45.176.1,58485,149.171.126.15,179,tcp,FIN,0.476639,470,354,254,...,0,8,8,2,2,2,1,2,Fuzzers,1


In [4]:
#finding & plotting graph of different attacks exists in dataset
labels, count = np.unique(dataset['attack_cat'].ravel(), return_counts = True)
height = count
bars = labels
y_pos = np.arange(len(bars))
plt.figure(figsize = (10, 3)) 
plt.bar(y_pos, height)
plt.xticks(y_pos, bars)
plt.xlabel("Attack Names")
plt.ylabel("Count")
plt.xticks(rotation=80)
plt.show()

NameError: name 'plt' is not defined

In [5]:
#graphs of different Crimes found in dataset
data = dataset.groupby(["srcip"])['attack_cat' ].count().sort_values(ascending=False).reset_index(name='Attack Count').reset_index()
data = data.iloc[0:20]
plt.figure(figsize=(9, 3))
sns.lineplot(data=data, x="srcip", y="Attack Count")
plt.xticks(rotation=70)
plt.title("Total Attacks Originates from Different Source IP")
plt.show()

NameError: name 'plt' is not defined

In [6]:
#applying dataset processing technique to convert non-numeric data to numeric data
labels = np.unique(dataset['attack_cat']).ravel()
dataset.drop(['Label'], axis = 1,inplace=True)
label_encoder = []
columns = dataset.columns
types = dataset.dtypes.values
for j in range(len(types)):
    name = types[j]
    if name == 'object': #finding column with object type
        le = LabelEncoder()
        dataset[columns[j]] = pd.Series(le.fit_transform(dataset[columns[j]].astype(str)))#encode all str columns to numeric
        label_encoder.append([columns[j], le])
dataset.fillna(dataset.mean(), inplace = True)    
dataset

NameError: name 'LabelEncoder' is not defined

In [7]:
#implementing Alert Templating & merging technique in which dataset will be reduce by merging same attack step
#port scan alert would include alerts corresponding to all ports being scanned by a single adversary
print("Dataset Size Before applying Merging Technique = "+str(dataset.shape[0]))
dataset = dataset.drop_duplicates(subset=['sport'], keep='first')#remove sam port rows which are scanned by same adversary
print("Dataset Size After applying Merging Technique = "+str(dataset.shape[0]))

Dataset Size Before applying Merging Technique = 22215
Dataset Size After applying Merging Technique = 9607


In [8]:
#implementing Alert Correlation and Alert Graph  which correlate relationships between generalized alerts.
#The correlation between alerts is measured using the correlation of their attributes. The alerts which have high correlation
#will be removed out to reduce graph size
correlation = dataset.T.corr().mean()#getting corr values using all attributes and rows
correlation = correlation.ravel()
index_list = dataset.index.to_list()
rows = []
for i in range(len(correlation)):
    if correlation[i] > 0.8:
        rows.append(index_list[i])
data = dataset.drop(index=rows)    
print("Dataset Size After applying Correlation Alert Graph = "+str(data.shape[0]))

Dataset Size After applying Correlation Alert Graph = 9607


In [9]:
#defining function to calculate mitre attack score for given attack technique id
def get_tactic_score(technique_id):    
    technique = mitre_attack_data.get_object_by_attack_id(technique_id, "attack-pattern")
    if technique:
        tactic_refs = technique.get("kill_chain_phases", [])
        tactic_scores = {}
        for tactic_ref in tactic_refs:
            tactic_name = tactic_ref["phase_name"]
            tactic_scores[tactic_name] = 1        
        # Add tactics with a score of 0 if the technique is not associated with them
        all_tactics = mitre_attack_data.get_tactics()
        for tactic in all_tactics:
            if tactic["name"] not in tactic_scores:
                tactic_scores[tactic["name"]] = 0                
        return tactic_scores
    else:
        return None
def getMitreScore(tactic_id):
    total = 0
    tactic_scores = get_tactic_score(tactic_id)
    if tactic_scores:        
        for tactic, score in tactic_scores.items():
            total += score
        if total > 1:
            total = total / len(tactic_scores.items())
    return total      

In [10]:
#implementing 3rd techique called  Alert Graph Partitioning to partition large graph into subgraph by dividing all distinct
#incidents in to smaller subgraph. Each subgraph will called Mitre attack with technique id to calculate incident score.
#This block implementing 3rd and 4th point
subgraph = []
mitre_tactic = ['T1550', 'T1556.007', 'T1078.001', 'T1556.001', 'T1574.002', 'T1098.002', 'T1078.003', 'T1556.004',
               'T1078.002', 'T1055.005']
mitre_score = []
incidents = np.unique(data['attack_cat'])
for i in range(len(incidents)):
    graph = data.loc[data['attack_cat'] == incidents[i]] #dividing large attack into smaller group using different incident APT 
    score = getMitreScore(mitre_tactic[i]) #for selected incident call Mitre function to calculate incident score
    graph['score'] = score #add score
    mitre_score.append(score)
    subgraph.append(graph) #and then generate subgraph with distinct incident
print("Total subgraph = "+str(len(subgraph)))  
print("Mitre Score Calculated for each incident = "+str(mitre_score))

NameError: name 'mitre_attack_data' is not defined

In [11]:
#now check resultant graph data with machine leanring algorithms to calculate accuracy
temp = pd.concat(subgraph)
Y = temp['attack_cat'].ravel()
Y = Y.astype(int)
temp.drop(['attack_cat'], axis = 1,inplace=True)
X = temp.values
#normalizing the graph data
scaler = StandardScaler()
X = scaler.fit_transform(X)
#split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print("Train & Test Dataset Split")
print("80% records used to train algorithms : "+str(X_train.shape[0]))
print("20% records features used to test algorithms : "+str(X_test.shape[0]))
data = np.load("model/data.npy", allow_pickle=True)
X_train, X_test, y_train, y_test = data

ValueError: No objects to concatenate

In [ ]:
#define global variables to save accuracy and other metrics
accuracy = []

In [ ]:
#function to calculate accuracy and other metrics
def calculateMetrics(algorithm, predict, y_test):
    a = accuracy_score(y_test,predict)*100
    accuracy.append(a)
    print(algorithm+" Accuracy  :  "+str(round(a, 3)))
    conf_matrix = confusion_matrix(y_test, predict) 
    plt.figure(figsize =(6, 3)) 
    ax = sns.heatmap(conf_matrix, xticklabels = labels, yticklabels = labels, annot = True, cmap="viridis" ,fmt ="g");
    ax.set_ylim([0,len(labels)])
    plt.title(algorithm+" Confusion matrix") 
    plt.xticks(rotation=90)
    plt.ylabel('True class') 
    plt.xlabel('Predicted class') 
    plt.show()  

In [ ]:
#training Random Forest ML algorithm on 80% training data and then evaluating performance on 20% test data
rf_cls = RandomForestClassifier()
#training on train data
rf_cls.fit(X_train, y_train)
#perfrom prediction on test data
predict = rf_cls.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("Random Forest", predict, y_test)

In [ ]:
#training AdaBoostClassifier ML algorithm on 80% training data and then evaluating performance on 20% test data
adaboost_cls = AdaBoostClassifier()
#training on train data
adaboost_cls.fit(X_train, y_train)
#perfrom prediction on test data
predict = adaboost_cls.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("AdaBoost", predict, y_test)

In [ ]:
#training Logistic Regression ML algorithm on 80% training data and then evaluating performance on 20% test data
lr_cls = LogisticRegression(max_iter=500)
#training on train data
lr_cls.fit(X_train, y_train)
#perfrom prediction on test data
predict = lr_cls.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("Logistic Regression", predict, y_test)

In [ ]:
#training KNN ML algorithm on 80% training data and then evaluating performance on 20% test data
knn_cls = KNeighborsClassifier(n_neighbors=2)
#training on train data
knn_cls.fit(X_train, y_train)
#perfrom prediction on test data
predict = knn_cls.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("KNN", predict, y_test)

In [ ]:
#training SVM Linear ML algorithm on 80% training data and then evaluating performance on 20% test data
svm_linear = svm.SVC(kernel="linear")
svm_linear.fit(X_train, y_train)
#perfrom prediction on test data
predict = svm_linear.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("SVM Linear", predict, y_test)

In [ ]:
#training SVM Linear ML algorithm on 80% training data and then evaluating performance on 20% test data
svm_rbf = svm.SVC(kernel="rbf")
svm_rbf.fit(X_train, y_train)
#perfrom prediction on test data
predict = svm_rbf.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("SVM RBF", predict, y_test)

In [ ]:
#training Decision Tree ML algorithm on 80% training data and then evaluating performance on 20% test data
dt_cls = DecisionTreeClassifier()
dt_cls.fit(X_train, y_train)
#perfrom prediction on test data
predict = dt_cls.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("Decision Tree", predict, y_test)

In [ ]:
#training MLP ML algorithm on 80% training data and then evaluating performance on 20% test data
mlp_cls = MLPClassifier(max_iter=500, hidden_layer_sizes=(10,), tol=0.01)
mlp_cls.fit(X_train, y_train)
#perfrom prediction on test data
predict = mlp_cls.predict(X_test)
#call this function to calculate accuracy and other metrics
calculateMetrics("MLP", predict, y_test)

In [ ]:
#training graphCNN algorithm
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)
#Create GNN model to detect fault from all services
graph_conv_filters = np.eye(1)
graph_conv_filters = K.constant(graph_conv_filters)
graph_model = Sequential()
graph_model.add(GraphCNN(64, 1, graph_conv_filters, input_shape=(X_train.shape[1],), activation='elu', kernel_regularizer=l2(5e-4)))
graph_model.add(GraphCNN(32, 1, graph_conv_filters, input_shape=(X_train.shape[1],), activation='elu', kernel_regularizer=l2(5e-4)))
graph_model.add(GraphCNN(1, 1, graph_conv_filters, input_shape=(X_train.shape[1],), activation='elu', kernel_regularizer=l2(5e-4)))
graph_model.add(Dense(units = 64, activation = 'elu'))
graph_model.add(Dense(units = y_train1.shape[1], activation = 'softmax'))
graph_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
if os.path.exists("model/gcnn_weights.h5") == False:
    hist = graph_model.fit(X_train, y_train1, batch_size=1, epochs=50, validation_data = (X_test, y_test1), verbose=1)
    graph_model.save_weights("model/gcnn_weights.h5")
else:
    graph_model.load_weights("model/gcnn_weights.h5")
#perform prediction on test data of all services and calculate accuracy and other metrics
pred = []
for i in range(len(X_test)):
    temp = []
    temp.append(X_test[i])
    temp = np.asarray(temp)
    predict = graph_model.predict(temp, batch_size=1)
    predict = np.argmax(predict)
    pred.append(predict)
y_tested = np.argmax(y_test1, axis=1)    
predict = np.asarray(pred)
#calling function to calculate accuracy on predicted data
calculateMetrics("Graph-CNN Model", y_tested, predict)

In [ ]:
#finding & plotting graph of different attacks exists in dataset
height = accuracy
bars = ['Random Forest', 'AdaBoost', 'Logistic Regression', 'KNN', 'SVM Linear', 'SVM RBF', 'Decision Tree', 'MLP', 
        'Extension Graph-CNN']
y_pos = np.arange(len(bars))
plt.figure(figsize = (6, 3)) 
plt.bar(y_pos, height)
plt.xticks(y_pos, bars)
plt.xlabel("Algorithm Names")
plt.ylabel("Accuracy")
plt.xticks(rotation=80)
plt.show()

In [ ]:
#display all algorithm performnace
data = []
for i in range(len(accuracy)):
    data.append([bars[i], accuracy[i]])
data = pd.DataFrame(data, columns=['Algorithm Name', 'Accuracy'])
data  

In [ ]:
#load test data and then predict incident type
test_dataset = pd.read_csv("Dataset/testData.csv")#reading test data
test = test_dataset.values
for i in range(len(label_encoder)):
    le = label_encoder[i]
    if le[0] != 'attack_cat':
        test_dataset[le[0]] = pd.Series(le[1].transform(test_dataset[le[0]].astype(str)))#encode all str columns to numeric
test_dataset.fillna(dataset.mean(), inplace = True)#replacing missing values with mean
test_dataset['score'] = mitre_score[0] #adding mitre score which will help algorithm in identfying APT attack
test_dataset = test_dataset.values
test_dataset = scaler.transform(test_dataset)#normalize test data
for i in range(len(test_dataset)):
    td = np.asarray([test_dataset[i]])
    predict = graph_model.predict(td, batch_size=1)#applying graph cnn extension model on test data to predict attack
    predict = np.argmax(predict)
    print("Test Data = "+str(test[i])+" ====> Predicted As : "+labels[predict])
    print()